# You will be working with basic Word2Vec Problems.

# Step 1: Load the Wikipedia GLoVE Word2Vec.


In [5]:
# importing required libraries
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.ensemble import GradientBoostingClassifier

import gensim
from gensim.models import Word2Vec

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/hims/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# ! wget --quite https://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
# import statments
import numpy
import re

'''
Tokenize each the sentences, example
Input : "John likes to watch movies. Mary likes movies too"
Ouput : "John","likes","to","watch","movies","Mary","likes","movies","too"
'''
def tokenize(sentences):
    words = []
    for sentence in sentences:
        w = word_extraction(sentence)
        words.extend(w)

    words = sorted(list(set(words)))
    return words

def word_extraction(sentence):
    ignore = ['a', "the", "is"]
    words = re.sub("[^\w]", " ",  sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text

def generate_bow(allsentences):
    vocab = tokenize(allsentences)
    print("Word List for Document \n{0} \n".format(vocab));

    for sentence in allsentences:
        words = word_extraction(sentence)
        bag_vector = numpy.zeros(len(vocab))
        for w in words:
            for i,word in enumerate(vocab):
                if word == w:
                    bag_vector[i] += 1

        print("{0} \n{1}\n".format(sentence,numpy.array(bag_vector)))


In [24]:
# importing all necessary modules
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec

# Reads ‘alice.txt’ file

data = []


for files in ["glove.6B.50d.txt", "glove.6B.100d.txt", "glove.6B.200d.txt", "glove.6B.300d.txt"]:
    sample = open(f"/Users/hims/Downloads/glove.6B/{files}", "r")
    s = sample.read()

    # Replaces escape character with space
    f = s.replace("\n", " ")

    # iterate through each sentence in the file
    for i in tqdm(sent_tokenize(f)):
        temp = []
        for j in word_tokenize(i):
            temp.append(j.lower())
        data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)



[nltk_data] Downloading package punkt to /Users/hims/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 983/983 [08:29<00:00,  1.93it/s]


# Step 2: Show how similar are these words:
    Man and Woman, Chair and Throne, water and baby
    

In [25]:
# Print results
print("Cosine similarity between 'alice' and 'wonderland' - CBOW : ",
    model1.wv.similarity('alice', 'wonderland'))

w1 = "machines"
print("Words similar to machine.")
model1.wv.most_similar(positive = w1)

Cosine similarity between 'alice' and 'wonderland' - CBOW :  0.12299221
Words similar to machine.


[('48.77', 0.45923206210136414),
 ('-0.00087209', 0.4538392126560211),
 ('quantico', 0.44871076941490173),
 ('ikazuchi', 0.44523826241493225),
 ('-0.00028789', 0.4446481764316559),
 ('-0.00027444', 0.4249381721019745),
 ('hims', 0.40548205375671387),
 ('6,070', 0.40199774503707886),
 ('glusac', 0.4015559256076813),
 ('sunuwar', 0.39948904514312744)]

In [26]:
word_pairs = [("man", "woman"), ("chair", "throne"), ("water", "baby")]

for word1, word2 in word_pairs:
    similarity = model1.wv.similarity(word1, word2)
    print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")

Similarity between 'man' and 'woman': -0.3469
Similarity between 'chair' and 'throne': 0.0636
Similarity between 'water' and 'baby': -0.1626


# Step 3: Using these provide analogies for the following:
    _____ is to King as Woman is to Man.
    _____ is to Princess as Man is to Woman.
    _____ is to a woman as a child is to an adult.

In [45]:
candidates = ['king', 'princess', 'woman']
for candidate in candidates:
    print(model1.wv.most_similar(candidate))
    
model.wv.most_similar(positive=['woman', 'king'], negative=['man'],restrict_vocab=1000000)

[('0.00027967', 0.46900370717048645), ('0.0007573', 0.4640876054763794), ('burroway', 0.4634557068347931), ('0.00045119', 0.4586903154850006), ('4.3082e-05', 0.4363987147808075), ('urvashi', 0.4355780780315399), ('kates', 0.4286333918571472), ('-1.2974e-05', 0.4274623990058899), ('aodong', 0.42041847109794617), ('0.00073753', 0.41819262504577637)]
[('-9.4602e-05', 0.4459092915058136), ('-0.00015746', 0.43822014331817627), ('-0.00042602', 0.4337364435195923), ('.193', 0.41566768288612366), ('-0.00018108', 0.4079437255859375), ('0.00051511', 0.3983939290046692), ('0.00097866', 0.3967643678188324), ('quiltmaking', 0.39591357111930847), ('-0.0007961', 0.3944344222545624), ('6.6492e-05', 0.39356961846351624)]
[('-0.00038437', 0.4794880747795105), ('sword-wielding', 0.43400412797927856), ('-8.5612e-05', 0.42903363704681396), ('robbin', 0.4282987415790558), ('-0.00069212', 0.4274161756038666), ('an-124', 0.4267188310623169), ('sonoita', 0.4245172441005707), ('0.00012724', 0.41801390051841736)

NameError: name 'model' is not defined

# Step 4: Apply Naive-Bayes Classifier on the Spam-Ham dataset shown in the demo.

In [5]:
! rm -rf spam.cs*
!wget --quiet https://raw.githubusercontent.com/anjali-ojha/deep-learning/ass-7/spam.csv

In [6]:
# loading sms data
data = pd.read_csv('https://raw.githubusercontent.com/anjali-ojha/deep-learning/ass-7/spam.csv ', encoding='latin-1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [7]:
data["v1"].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


X = data
# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['v2'], data['v1'], test_size=0.2, random_state=2024)

# Step 3: Vectorize the text data using CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)
print(X_train_counts.shape)

(4457, 7742)


In [20]:
# pre processing text
def text_pre_processing(sms):

    # removing punctuations
    sms_wo_punct = [x for x in sms if x not in string.punctuation]
    sms_wo_punct = ''.join(sms_wo_punct)

    # keeping only english letters
    sms_wo_punct_only_words = [x for x in sms_wo_punct if x.isalpha() or x == " "]
    sms_wo_punct_only_words = ''.join(sms_wo_punct_only_words)

    # converting the data to lower case
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words.lower()

    # removing stopwords
    sms_wo_punct_only_words_lc = sms_wo_punct_only_words_lc.split(" ")
    sms_wo_punct_only_words_lc_wo_sw = [x for x in sms_wo_punct_only_words_lc if x not in stopwords.words('english')]
    sms_wo_punct_only_words_lc_wo_sw = ' '.join(sms_wo_punct_only_words_lc_wo_sw)

    return sms_wo_punct_only_words_lc_wo_sw

In [21]:
data_processed = pd.DataFrame(columns = ["pre_processed_sms"])
for i in tqdm(data["v2"]):
    data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
data_processed

Streaming output truncated to the last 5000 lines.
<ipython-input-21-a9bbfef49a97>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-21-a9bbfef49a97>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-21-a9bbfef49a97>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_processed = data_processed.append({"pre_processed_sms": text_pre_processing(i)}, ignore_index = True)
<ipython-input-21-a9bbfef49a97>:3: FutureWarning: The frame.append method is deprecated and will be removed from panda

,pre_processed_sms
0,go jurong point crazy available bugis n great ...
1,ok lar joking wif u oni
2,free entry wkly comp win fa cup final tkts st...
3,u dun say early hor u c already say
4,nah dont think goes usf lives around though
...,...
5567,nd time tried contact u u å pound prize clai...
5568,ì b going esplanade fr home
5569,pity mood soany suggestions
5570,guy bitching acted like id interested buying s...


In [22]:
# Splitting into train and test
x_train, x_test, y_train, y_test = train_test_split(data_processed, data["v1"], test_size = 0.20, random_state = 100)
x_train = x_train.reset_index(drop = True)
x_test = x_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [24]:
# Using word to vec
alg = Word2Vec(500, min_count = 1)
alg.build_vocab(x_train)
alg.train(x_train, total_examples = len(x_train), epochs = alg.epochs)

TypeError: The corpus_iterable must be an iterable of lists of strings, got 500 instead

In [16]:
# Step 4: Train a Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_counts, y_train)

# Step 5: Evaluate the classifier
y_pred = nb_classifier.predict(X_test_counts)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report for detailed evaluation
print(classification_report(y_test, y_pred))

Accuracy: 0.9874
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       977
        spam       0.98      0.92      0.95       138

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

